[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LLaMA-VID-jupyter/blob/main/LLaMA_VID_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaMA-VID
%cd /content/LLaMA-VID

!pip install -q -e .
!pip install -q ninja
!pip install -q flash-attn --no-build-isolation

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://storage.googleapis.com/sfr-vision-language-research/LAVIS/models/BLIP2/eva_vit_g.pth -d /content/model_zoo/LAVIS -o eva_vit_g.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/config.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer_config.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/vocab.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/special_tokens_map.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/preprocessor_config.json -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o preprocessor_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/model_zoo/OpenAI/clip-vit-large-patch14 -o pytorch_model.bi

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python', '-m', 'llamavid.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()
# !python -m llamavid.serve.controller --host 0.0.0.0 --port 10000

In [ ]:
import threading
import subprocess
command = [
    'python', '-m', 'llamavid.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '4bit/llama-vid-7b-full-224-long-video-5GB',
    '--load-8bit'
]
threading.Thread(target=lambda: subprocess.run(command, check=True, shell=False), daemon=True).start()
# !python -m llamavid.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path 4bit/llama-vid-7b-full-224-long-video-5GB

In [ ]:
!python -m llamavid.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share